In [1]:
import pandas as pd
from pandas import Series, DataFrame
import matplotlib
matplotlib.rcParams.update({'font.size':15})
matplotlib.rcParams.update({'figure.figsize': [6, 6]})
%matplotlib inline

all_data = pd.read_csv('s19-city/data/spend.csv');
print(all_data["2018"].abs().sum())
streets_dataset = all_data[all_data["Agency"].str.startswith("44")]

#print(streets_dataset);
#print(f"Number of data points: {len(city_data)}")

352419540.5


In [2]:
# Billing: Expense, with parens, all negative
int_billing = all_data[all_data["Character Code"].str.startswith("58")]
# Charges: Also expense, sometimes has parens, mostly positive but a few negative
int_charges = all_data[all_data["Character Code"].str.startswith("57")]
charges_tot = int_charges["2018"].sum()
charges_tot = int_charges["2018"].abs().sum()
print("Net charges: " + str(charges_tot) + " Total charges: " +  str(charges_tot_abs))
billing_tot = int_billing["2018"].abs().sum()
print(int_billing["2018"].abs().sum())
df = Series({"Billing":int_billing["2018"].abs().sum()/1000000, "Charges":int_charges["2018"].abs().sum()/1000000})
ax = df.plot.bar(figsize = (10, 10))
ax.set_ylabel("Expense in Millions of Dollars")
ax.set_xlabel("")
ax.set_title("Total Interdepartmental Billing and Charges")




NameError: name 'charges_tot_abs' is not defined

In [3]:
int_billing["2018"].abs().sum()

3047656.06

In [4]:
# Creates list of agencies
agencies = all_data["Agency"].str[6:21].unique()
print(agencies)

['LIBRARY' 'PARKS' 'STREETS' 'PLANNING' 'MUNICIPAL COURT' 'FIRE'
 'ENGINEERING' 'UNDEFINED' 'POLICE' 'TRAFFIC ENGINEE' 'HUMAN RESOURCES'
 'INFORMATION TEC' 'MAYOR' 'COMMUNITY DEVEL' 'BUILDING INSPEC' 'ATTORNEY'
 'FINANCE' 'COMMON COUNCIL' 'ECONOMIC DEVELO' 'CLERK' 'TREASURER'
 'PCED OFFICE OF ' 'ASSESSOR' 'CIVIL RIGHTS' 'TRANSPORTATION'
 'EMPLOYEE ASSIST' 'CITY WIDE MISCE']


In [5]:
# All data
charges_multilevel = int_charges.copy()
charges_multilevel["Agency"] = charges_multilevel["Agency"].str[6:]
charges_multilevel["Object"] = charges_multilevel["Object"].str[18:]

charges_multilevel = charges_multilevel.groupby(["Agency", "Object"]).sum().abs()["2018"]
charges_multilevel = charges_multilevel[charges_multilevel != 0]

top_agencies = charges_multilevel.copy()
top_agencies = top_agencies.filter(regex='FIRE|POLICE|PARKS|STREETS', axis=0)

other = charges_multilevel.copy()
other = other.filter(regex = '^((?!(FIRE|STREETS|POLICE|PARKS)).)*$', axis = 0)
other = other.rename(lambda s:"Other", level=0 ).groupby(["Agency", "Object"]).sum()

result = other.append(top_agencies)
result = result.rename(lambda s:s.title())
result = result.sort_values(ascending = False)
#charges_multilevel = charges_multilevel[charges_multilevel >= 150000]
print(result)

result.to_csv("~/data2.csv")


#print(charges_multilevel)
#objects = billing.groupby([column2])["2018"].sum().abs().sort_values(ascending=False).head(10)

Agency   Object              
Streets  From Fleet Services     7331039.25
Fire     From Fleet Services     2627818.02
Police   From Fleet Services     2407865.01
Parks    From Fleet Services     1885951.60
Police   From Insurance           998518.00
Fire     From Workers Comp        859647.00
Police   From Workers Comp        835092.00
         From Engineering         611303.00
Streets  From Workers Comp        398717.00
Other    From Engineering         388705.07
         From Fleet Services      310283.72
         From Insurance           281298.00
Fire     From Engineering         278163.00
Police   From Traffic Enginee     273847.41
Parks    From Workers Comp        273306.00
Other    From Workers Comp        272793.00
Parks    From Insurance           176555.00
Fire     From Traffic Enginee     120575.71
Streets  From Insurance           115286.00
Fire     From Insurance           101277.00
Other    From Community Devel      90000.00
Streets  From Engineering          55152.00
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [ ]:
# data between agencies for charges
charge = int_charges.copy()
charge["Agency"] = charge["Agency"].str[6:21]
charge["Object"] = charge["Object"].str[23:]

charge = charge[charge["Object"].isin(agencies)]

charge = charge.groupby(["Agency", "Object"]).sum()["2018"]
charge = charge[charge != 0]

charge = charge.rename(lambda s:s.title())

#print(charge)

# Data between agencies for billing
bill = int_billing.copy()
bill["Agency"] = bill["Agency"].str[6:21]
bill["Object"] = bill["Object"].str[22:]


bill = bill[bill["Object"].isin(agencies)]

bill = bill.groupby(["Agency", "Object"]).sum()["2018"]
bill = bill[bill != 0]

bill = bill.rename(lambda s:s.title())

print(bill)
print(charge)

bill.abs().to_csv("~/Projects/DataAnalysis/billing_agency_only.csv")

charge.abs().to_csv("~/Projects/DataAnalysis/charges_agency_only.csv")


no_descrep = 0
for i in charge.items():
    agency = i[0][0]
    obj = i[0][1]
    ag1 = charge[agency][obj]
    try:
        ag2 = bill[obj][agency]
    except:
        ag2 = 0
    dif = (int)(ag1 + ag2)
    if (dif != 0):
        print(agency + " => "   + obj)
        print("\tCharge: $" + str(int(ag1))+ " Billing: $" + str(int(ag2)))
        print("\tDiscrepancy $" + str(dif) + " \n")
    else: 
        no_descrep += 1
print("Combinations without discrepancies: " + str(no_descrep))

In [ ]:
charge = int_charges.copy()
charge["Agency"] = charge["Agency"].str[6:21]
charge["Object"] = charge["Object"].str[23:]
charge = charge[charge["Object"].str.contains("TRAFFIC") ]
charge = charge[charge["2018"] != 0]

# Data between agencies for billing
bill = int_billing.copy()
bill["Agency"] = bill["Agency"].str[6:21]
bill["Object"] = bill["Object"].str[22:]
bill = bill[bill["Agency"].str.contains("TRAFFIC")]
bill = bill[bill["2018"] != 0]

print(bill.sort_values(ascending = False, by = "2018")[["Agency", "Object", "2018"]])
print(charge.sort_values(by = "2018")[["Agency", "Object", "2018"]])


In [ ]:

for i in bill.itertuples():
    # 9 is for 2018
    # print(i[9])
    found = False
    for j in charge.itertuples(): 
        if(i[9] == j[9]):
            found = True
    if(found):
        print(i[9])



In [ ]:
revenue_dataset = streets_dataset[streets_dataset["Account Type"] == "Revenue"]
expense_dataset = streets_dataset[streets_dataset["Account Type"] == "Expense"]

tot_revenue = revenue_dataset["2018"].abs().sum() / 10 **6
tot_expense = expense_dataset["2018"].abs().sum() / 10**6

label_column = Series(["Revenue", "Expenses"])
series_column = Series([tot_revenue, tot_expense])

table = DataFrame({"type":label_column, "Cost (in millions)":series_column})
ax = table.plot.bar(x = 'type', y = 'Cost (in millions)', legend = False)
ax.set_xlabel("")
ax.set_ylabel("Cost in millions")
ax.set_title("Revenue and Expenses")

In [ ]:
expenses_by_type = expense_dataset.groupby(["Character Code"])
expense_types = expenses_by_type["2018"].sum()
ax = expense_types.plot.pie(figsize = (6,6))
ax.set_ylabel("")
ax.set_title("Expense Sources of Streets")


In [ ]:
interdep_billing = expense_dataset[expense_dataset["Character Code"] == '58 - INTER DEPART BILLING']
objects = interdep_billing.groupby(["Object"])["2015"].sum().abs()
#series1 = objects.tail(2)
#other = objects.sum() - series1.sum()
#series2 = Series([other], index=['Other'])

#plot_data = pd.concat(objects)
ax2 = objects.plot.pie(figsize=(6,6), autopct='%d%%')
ax2.set_ylabel("")
ax2.set_title("Interdepartmental Billing for Streets")

In [ ]:
interdep_charges = expense_dataset[expense_dataset["Character Code"] == '57 - INTER DEPART CHARGES']
objects = interdep_charges.groupby(["Object"])["2018"].sum().sort_values()

series1 = objects.tail(2)
other = objects.sum() - series1.sum()
series2 = Series([other], index=['Other'])

plot_data = pd.concat([series1, series2])
ax3 = plot_data.plot.pie(figsize=(6,6), autopct='%d%%')
ax3.set_ylabel("")
ax3.set_title("Interdepartmental Charges for Streets")

In [ ]:
revenue_types = revenue_dataset.groupby(["Character Code"])["2018"].sum().abs()
ax = revenue_types.plot.pie()
ax.set_ylabel("")
ax.set_title("Revenue Sources")

In [ ]:
total_revenue_all_years = Series([revenue_dataset["2015"].sum(), revenue_dataset["2016"].sum(), revenue_dataset["2017"].sum(), revenue_dataset["2018"].sum()]).abs() / 10**6
df = DataFrame({"Year": Series([2015, 2016, 2017, 2018]), "Revenue (millions)": total_revenue_all_years})
ax = df.plot.line(x="Year", y= "Revenue (millions)",ylim=[0, total_revenue_all_years.max() * 1.5], legend=False, figsize = (12, 6))
ax.set_ylabel("Millions of dollars")
ax.set_title("Revenue over time")
ax.set_xticks([2015, 2016, 2017, 2018])

In [ ]:
total_expenses_all_years = Series([expense_dataset["2015"].sum(), expense_dataset["2016"].sum(), expense_dataset["2017"].sum(), expense_dataset["2018"].sum()]).abs() / 10**6
df = DataFrame({"Year": Series([2015, 2016, 2017, 2018]), "Revenue (millions)": total_expenses_all_years})
ax = df.plot.line(x="Year", y= "Revenue (millions)",ylim=[0, total_expenses_all_years.max() * 1.5], legend=False, figsize = (12,6))
ax.set_ylabel("Millions of dollars")
ax.set_title("Expenses over time")
ax.set_xticks([2015, 2016, 2017, 2018])

In [ ]:
objects = int_billing.groupby(["Object"])["2018"].sum().abs().sort_values(ascending = False)/1000
objects = objects.rename(lambda s:s[19:] )
ax2 = objects.plot.bar(figsize=(10,6))
ax2.set_ylabel("Expense in Thousands of Dollars")
ax2.set_xlabel("Object")
ax2.set_title("Interdepartmental Billing by Object")

In [ ]:
objects = int_charges.groupby(["Object"])["2018"].sum().abs().sort_values(ascending=False).head(5)
# Todo: add misc.
objects = objects.rename(lambda s:s[17:] )

ax2 = objects.plot.pie(figsize=(6,6), autopct='%d%%')
ax2.set_ylabel("")
ax2.set_title("Interdepartmental Charges by Object")

In [ ]:
# Agencies that have the most interdepartmental billing
objects = int_billing.groupby(["Agency"])["2018"].sum().abs().sort_values(ascending=False)
# Todo: add misc.
objects = objects.rename(lambda s:s[5:] )

ax2 = objects.plot.pie(figsize=(6,6), autopct='%d%%')
ax2.set_ylabel("")
ax2.set_title("Interdepartmental Billing by Agency")

In [ ]:
# Agencies that have the most interdepartmental charges
objects = int_charges.groupby(["Agency"])["2018"].sum().abs().sort_values(ascending=False).head(6)
# Todo: add misc.
objects = objects.rename(lambda s:s[5:] )

ax2 = objects.plot.pie(figsize=(6,6), autopct='%d%%')
ax2.set_ylabel("")
ax2.set_title("Interdepartmental Charges by Agency")

In [ ]:
objects = int_billing.groupby(["Organization"])["2018"].sum().abs().sort_values(ascending = False).head(8)/1000
objects = objects.rename(lambda s:s[8:] )
ax2 = objects.plot.bar(figsize=(6,6))
ax2.set_ylabel("Millions of Dollars")
ax2.set_xlabel("Object")
ax2.set_title("Interdepartmental Billing by Org")

In [ ]:
objects = int_charges.groupby(["Organization"])["2018"].sum().abs().sort_values(ascending = False).head(30)/1000000
objects = objects.rename(lambda s:s[8:] )
ax2 = objects.plot.bar(figsize=(12,6))
ax2.set_ylabel("Millions of Dollars")
ax2.set_xlabel("Object")
ax2.set_title("Interdepartmental Charges by Org")

In [7]:
def graph_charges(column1, value, column2, trim=None):
    # Isolates only data points in column1 matching "value",
    # Then graphs them grouped by column2.
    # Trims the first trim characters off of the labels
    if(trim == None):
        trim = 0
    # obj is string
    billing = int_charges[int_charges[column1].str.contains(value)]
    objects = billing.groupby([column2])["2018"].sum().abs().sort_values(ascending=False).head(10)
    # Todo: add misc.
    objects = objects.rename(lambda s:s[trim:] )

    ax2 = objects.plot.pie(figsize=(6,6), autopct='%d%%')
    ax2.set_ylabel("")
    ax2.set_title("Charges in " + column1.lower() + " " + value.lower() + " by " + column2.lower())
    return ax2


In [ ]:
def graph_billing(column, org, column2, trim=None):
    if trim == None:
        trim = 0
    # obj is string
    billing = int_billing[int_billing[column].str.contains(org)]
    objects = billing.groupby([column2])["2018"].sum().abs().sort_values(ascending=False).head(10)
    # Todo: add misc.
    objects = objects.rename(lambda s:s[trim:] )

    ax2 = objects.plot.pie(figsize=(6,6), autopct='%d%%')
    ax2.set_ylabel("")
    ax2.set_title("Billing in " + column.lower() + " " + org.lower() + " by " + column2.lower() )
    return ax2


In [ ]:
#1 
graph_billing("Agency", "40 -  ENGINEERING", "Object", 18)

In [ ]:
#1 
graph_billing("Agency", "40 -  ENGINEERING", "Organization", 7)

In [ ]:
#2
graph_billing("Agency", "TRAFFIC ENGINEERING", "Object", 18)

In [ ]:
#2
graph_billing("Agency", "TRAFFIC ENGINEERING", "Organization", 7)

In [ ]:
#3
graph_billing("Agency", "INFORMATION TECHNOLOGY", "Object")

In [ ]:
#3
graph_billing("Agency", "INFORMATION TECHNOLOGY", "Organization", 7)

In [ ]:
graph_charges("Agency","STREETS", "Object", 18)

In [ ]:
graph_charges("Agency","STREETS", "Organization", 7)

In [ ]:
graph_charges("Agency","POLICE", "Object", 18)

In [ ]:
graph_charges("Agency","POLICE", "Organization", 7)

In [ ]:
graph_charges("Agency","FIRE", "Object", 18)

In [ ]:
graph_charges("Agency","FIRE", "Organization", 7)

In [ ]:
graph_billing("Object", "FLEET SERVICES", "Agency", 5)

In [ ]:
graph_charges("Object", "FLEET SERVICES", "Agency", 5)